# OpenCellID Data Exploration with Pandas (accelerated by cudf.pandas)

[OpenCellID](https://wiki.opencellid.org/wiki/What_is_OpenCellID) is the world's largest collaborative community project that collects GPS positions of cell towers, used free of charge, for a multitude of commercial and private purposes.

The OpenCellID project was primarily created to serve as a data source for GSM localisation. As of October, 2017, the database contained almost 36 million unique GSM Cell IDs. More than 75,000 contributors have already registered with OpenCellID, contributing millions of new measurements every day in average to the OpenCellID database.

OpenCellID provides 100% free Cell ID data under a [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/). The OpenCellID database is published under an open content license with the intention of promoting free use and redistribution of the data. All data uploaded by any of the contributors can also be downloaded again free of charge - no exceptions!



## Load cudf.pandas extension

In [ ]:
%load_ext cudf.pandas

## Import required Packages

In [ ]:
# importing viz libraries
import hvplot.pandas
import pydeck as pdk
import panel as pn
pn.extension("deckgl", loading_indicator=True, template='material')

#importing pandas
import pandas as pd

## Download cell-data (https://www.opencellid.org/)

> Google Colab: Use download_us_dataset() for a smaller dataset to test the notebook with the free tier of google colab

In [ ]:
# Download the script from a GitHub repository
# !wget https://raw.githubusercontent.com/your-username/your-repo/main/opencellid_downloader.py

from opencellid_downloader import OpenCellIDDownloader
OpenCellIDDownloader(token="your_token_here").download_us_dataset() # use download_full_dataset() to download the entire world dataset

## Read cell-data (https://www.opencellid.org/)

### Column Descriptions

- **Radio**:	The generation of broadband cellular network technology (Eg. LTE, GSM)

- **MCC**:	Mobile country code. This info is publicly shared by International Telecommunication Union (link)

- **MNC**:	Mobile network code. This info is publicly shared by International Telecommunication Union (link)

- **AREA**:	Location Area Code

- **CELL**:	This is a unique number used to identify each Base transceiver station or sector of BTS

- **Longitude**:	Longitude, is a geographic coordinate that specifies the east-west position of a point on the Earth's surface

- **Latitude**:	Latitude is a geographic coordinate that specifies the north–south position of a point on the Earth's surface.

- **Range**:	Approximate area within which the cell could be. (In meters)

- **Samples**:	Number of measures processed to get a particular data point

- **Changeable=1**:	The location is determined by processing samples

- **Changeable=0**:	The location is directly obtained from the telecom firm

- **Created**:	When a particular cell was first added to database (UNIX timestamp)

- **Updated**:	When a particular cell was last seen (UNIX timestamp)

- **AverageSignal**:	To get the positions of cells, OpenCelliD processes measurements from data contributors. Each measurement includes GPS location of device + Scanned cell identifier (MCC-MNC-LAC-CID) + Other device properties (Signal strength). In this process, signal strength of the device is averaged. Most ‘averageSignal’ values are 0 because OpenCelliD simply didn’t receive signal strength values.


In [ ]:
%%cudf.pandas.profile
df = pd.read_csv('./opencellid_data/cell_towers_us.csv')

df.rename(columns={'net':'MNC', 'mcc':'MCC'}, inplace=True)

print(df.shape)

In [ ]:
df.head()

## Read mcc-mnc dataset to map to carrier name

Publicaly available dataset: https://mcc-mnc.net/ powered by [simbase](https://www.simbase.com/)


In [ ]:
%%time
df_carriers = pd.read_csv('https://s3.amazonaws.com/mcc-mnc.net/mcc-mnc.csv', sep=';')[['MCC', 'MNC', 'Country', 'Operator']]

# rename Operator to Operator
df_carriers.rename(columns={'Operator': 'Operator'}, inplace=True)

In [ ]:
df_carriers.head()

In [ ]:
%%time
df_final = df.merge(df_carriers, left_on=['MCC', 'MNC'], right_on=['MCC', 'MNC']).drop(columns=['MCC', 'MNC'])

In [ ]:
%%time
df_final.head()

### Analyzing Operator types

In [ ]:
%%time
# Most reported cell towers by operators
df_final.Operator.value_counts()

In [ ]:
%%time
# Categorizing Operators with less than 1000 cells to other to reduce clutter
operatorss_to_rename = df_final.Operator.value_counts()[df_final.Operator.value_counts()<=1000].index.tolist()
len(operatorss_to_rename)

In [ ]:
%%time
df_final['Operator'] = df_final['Operator'].replace(operatorss_to_rename, 'Other')

In [ ]:
operators_df = df_final.groupby(['Country', 'Operator']).agg({'radio':'count'}).reset_index()

In [ ]:
# Final operator analysis
operators_df

### Analyzing Radio types

In [ ]:
# Analyzing radio cell tower types
df_final.radio.value_counts().reset_index().hvplot.barh(x='radio')

In [ ]:
# Most LTE towers country wise
df_final[df_final.radio == 'LTE'].Country.value_counts().reset_index().head(10).hvplot.barh(x='Country')

In [ ]:
%%time
columns = ['radio', 'area', 'lon', 'lat', 'range', 'samples', 'changeable', 'created', 'averageSignal', 'Country', 'Operator']
df_final = df_final[columns]

## Create an exploratory panel dashboard

In [ ]:
%%time
# Convert 'radio' to a color value to display in the dashboard
color_map = {
    'UMTS': [255, 165, 0, 140],  # Orange
    'LTE': [255, 0, 0, 140],     # Red
    'GSM': [135, 206, 250, 140], # Light Blue
    'CDMA': [123, 104, 238, 140],# Medium Slate Blue
    'NR': [0, 128, 0, 140]       # Green
}
df_final['color'] = df_final['radio'].map(color_map)

In [ ]:
# declare a legend
legend_markdown = """
### Radio Type Legend

- **UMTS**: <span style="height: 10px; width: 10px; background-color: rgba(255, 165, 0, 0.55); border-radius: 50%; display: inline-block;"></span>
- **LTE**: <span style="height: 10px; width: 10px; background-color: rgba(255, 0, 0, 0.55); border-radius: 50%; display: inline-block;"></span>
- **GSM**: <span style="height: 10px; width: 10px; background-color: rgba(135, 206, 250, 0.55); border-radius: 50%; display: inline-block;"></span>
- **CDMA**: <span style="height: 10px; width: 10px; background-color: rgba(123, 104, 238, 0.55); border-radius: 50%; display: inline-block;"></span>
- **NR**: <span style="height: 10px; width: 10px; background-color: rgba(0, 128, 0, 0.55); border-radius: 50%; display: inline-block;"></span>
"""

In [ ]:
# point_budget is used to curtail number of points rendered at any time. Can be adjusted as per client browser capabilities
point_budget = pn.widgets.IntSlider(name="Point Budget", value=100_000, start=10_000, end=1_000_000, step=10_000)

# country to display the cell-tower data points for
countries = pn.widgets.Select(name="Countries", value="United States of America", options=df_final.Country.unique().tolist())

# filter by radio cell tower type (GSM, LTE, UMTS, CDMA or all)
radio = pn.widgets.Select(name="Radio Tower Type", value="all", options=df_final.radio.unique().tolist() + ['all'])

In [ ]:
pdk.settings.custom_libraries = [
    {
        "libraryName": "MyTileLayerLibrary",
        "resourceUri": "https://cdn.jsdelivr.net/gh/agressin/pydeck_myTileLayer@master/dist/bundle.js",
    }
]

DATA_URL = 'https://c.tile.openstreetmap.org/{z}/{x}/{y}.png'

custom_layer = pdk.Layer(
    "MyTileLayer",
    DATA_URL
)
# Interactive plot function that displays a globe view, and cell-tower datapoints on top in deckgl
def plot(point_budget, country, radio='all'):
    selected_indices = df_final.Country == country
    if radio != 'all':
        selected_indices &= df_final.radio == radio
    df_visualize = df_final[selected_indices]

    if point_budget < df_visualize.shape[0]:
        df_visualize = df_visualize.sample(n=point_budget)
    COUNTRIES = "https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_50m_admin_0_scale_rank.geojson"
    # Define the layer
    layers = [
        pdk.Layer(
            "GeoJsonLayer",
            id="base-map",
            data=COUNTRIES,
            stroked=False,
            filled=True,
            get_fill_color=[200, 200, 200],
        ),
        pdk.Layer(
        'ScatterplotLayer',  # Use ScatterplotLayer
        df_visualize,
        pickable=True,
        opacity=0.8,
        stroked=True,
        filled=True,
        radius_scale=6,
        radius_min_pixels=1,
        radius_max_pixels=10,
        line_width_min_pixels=1,
        get_position=['lon', 'lat'],
        radius=1000,
        get_line_color="color"
    )]
    # Set the viewport location
    view_state = pdk.ViewState(latitude=df_visualize.lat.median(), longitude=df_visualize.lon.median(), zoom=2, bearing=0, pitch=0)
    view = pdk.View(type="_GlobeView", controller=True, width=1000, height=700)
    # Render
    return pn.pane.DeckGL(pdk.Deck(views=[view],layers=layers, initial_view_state=view_state, map_provider=None,
            ), height=600, width=1200)

# Current dataSize as per the active filters
def dataSize(country, radio):
    selected_indices = df_final.Country == country
    if radio != 'all':
        selected_indices &= df_final.radio == radio
    return pn.widgets.Number(name="Data Size", value=df_final[selected_indices].shape[0], format="{value:,}")


# Bind above interactive plots to pn.Widgets declared above
interactive_plot = pn.bind(plot, point_budget, countries, radio)
datasize_plot = pn.bind(dataSize, countries, radio)

In [ ]:
# Declare the layout of the dashboard and run
pn.Row(
    pn.WidgetBox(
        datasize_plot,
        point_budget,
        countries,
        radio,
        pn.pane.Markdown(legend_markdown)
    ),
    interactive_plot
)